In [1]:
from datasets import load_dataset, DatasetDict, Dataset, load_from_disk
import pandas
from tqdm import tqdm

In [2]:
dataset_name = "presencesw/multinli"
label = 'entailment'
dataset = load_dataset(dataset_name)

In [3]:
# dataset['train'][:10]

In [4]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['gold_label', 'sentence1', 'sentence2'],
        num_rows: 392702
    })
    dev_matched: Dataset({
        features: ['gold_label', 'sentence1', 'sentence2'],
        num_rows: 9815
    })
    dev_mismatched: Dataset({
        features: ['gold_label', 'sentence1', 'sentence2'],
        num_rows: 9832
    })
})


In [5]:
def convert_entailment(exmaples):
    labels = exmaples['gold_label']
    labels = [f"not_{label}" if i != label else label for i in labels]
    exmaples['gold_label'] = labels
    return exmaples

In [6]:
# this step convert dataset to 2 labels
converted_dataset = dataset.map(convert_entailment, batched=True)

# check number samples in dataset train
converted_dataset['train']

Dataset({
    features: ['gold_label', 'sentence1', 'sentence2'],
    num_rows: 392702
})

In [7]:
# Divine the dataset to 2 sets `entailment` and `not entailment`
new_dataset_entail = converted_dataset.filter(lambda example: example['gold_label'] == label)['train']
new_dataset_not_entail = converted_dataset.filter(lambda example: example['gold_label'] == f'not_{label}')['train']

In [8]:
# convert dataset to pandas for next step
new_dataset_entail = Dataset.to_pandas(new_dataset_entail)
# duplicate dataset not entailment
new_dataset_not_entail1 = Dataset.to_pandas(new_dataset_not_entail)
new_dataset_not_entail2 = Dataset.to_pandas(new_dataset_not_entail)

In [9]:
# merge 2 dataset not entailment for the target
j = 0
for i in tqdm(range(len(new_dataset_not_entail1))):
    temp = set()
    # check = False
    # for j in range(len(new_dataset_not_entail2)):
    for index in range(100):
        new_j = j + index
        if new_j == len(new_dataset_not_entail1):
            j = new_j
            break
        if new_dataset_not_entail1.iloc[i]['sentence1'] != new_dataset_not_entail2.iloc[new_j]['sentence1']:
            j = new_j
            break
        temp.add(new_dataset_not_entail2.iloc[new_j]['sentence2'])
    # new_dataset_not_entail1.iloc[i]['sentence2'] = temp
    # temp = [val for val in temp]
    new_dataset_not_entail1.at[i, 'sentence2'] = temp

  0%|          | 0/261803 [00:00<?, ?it/s]

100%|██████████| 261803/261803 [00:25<00:00, 10292.51it/s]


In [10]:
# dataset_not_entail = new_dataset_not_entail1.drop_duplicates(subset=['sentence2'])
# print(len(dataset_not_entail))
dataset_not_entail = new_dataset_not_entail1[new_dataset_not_entail1['sentence2'] != set()]
print(len(dataset_not_entail))

261799


In [11]:
dataset_not_entail = dataset_not_entail.reset_index(drop=True)
dataset_not_entail

,gold_label,sentence1,sentence2
0,not_entailment,Conceptually cream skimming has two basic dime...,{Product and geography are what make cream ski...
1,not_entailment,yeah i tell you what though if you go price so...,{The tennis shoes have a range of prices.}
2,not_entailment,But a few Christian mosaics survive above the ...,{Most of the Christian mosaics were destroyed ...
3,not_entailment,Gays and lesbians.,{Heterosexuals.}
4,not_entailment,At the end of Rue des Francs-Bourgeois is what...,{Place des Vosges is constructed entirely of g...
...,...,...,...
261794,not_entailment,"Legacy, predicting that he will keep custody o...",{His children love their father and it's a gre...
261795,not_entailment,"Clearly, California can - and must - do better.",{California cannot do any better.}
261796,not_entailment,It was once regarded as the most beautiful str...,{So many of the original buildings had been re...
261797,not_entailment,Obituaries fondly recalled his on-air debates ...,{The obituaries were beautiful and written in ...


In [12]:
new_dataset_entail

,gold_label,sentence1,sentence2
0,entailment,you know during the season and i guess at at y...,You lose the things to the following level if ...
1,entailment,One of our number will carry out your instruct...,A member of my team will execute your orders w...
2,entailment,How do you know? All this is their information...,This information belongs to them.
3,entailment,my walkman broke so i'm upset now i just have ...,I'm upset that my walkman broke and now I have...
4,entailment,(Read for Slate 's take on Jackson's findings.),Slate had an opinion on Jackson's findings.
...,...,...,...
130894,entailment,Dolly Parton's breast implants against Mark Mc...,Dolly Parton's breast implants contrasting Mar...
130895,entailment,The New Radicals sound like Todd Rundgren has ...,The New Radicals sound like Todd Rundgren.
130896,entailment,If you would like to scale the highest heights...,Some lessons and advice are a good idea if you...
130897,entailment,Drawing a blank?,You can be drawing a blank.


In [13]:
dataset_not_entail = dataset_not_entail.rename(columns={"sentence2": "negative"})
dataset_not_entail = dataset_not_entail.drop(columns="gold_label")

In [14]:
new_dataset_entail = new_dataset_entail.merge(dataset_not_entail, how="left", left_on="sentence1", right_on="sentence1")

In [15]:
new_dataset_entail

,gold_label,sentence1,sentence2,negative
0,entailment,you know during the season and i guess at at y...,You lose the things to the following level if ...,{They never perform recalls on anything.}
1,entailment,you know during the season and i guess at at y...,You lose the things to the following level if ...,{The recalls are done every Friday.}
2,entailment,One of our number will carry out your instruct...,A member of my team will execute your orders w...,{We have no one free at the moment so you have...
3,entailment,One of our number will carry out your instruct...,A member of my team will execute your orders w...,{You provided concise instructions with clear ...
4,entailment,How do you know? All this is their information...,This information belongs to them.,{We are not sure if they are correct.}
...,...,...,...,...
274822,entailment,If you would like to scale the highest heights...,Some lessons and advice are a good idea if you...,{Climbing is a good sport for novices. }
274823,entailment,Drawing a blank?,You can be drawing a blank.,{You aren't drawing a blank.}
274824,entailment,Drawing a blank?,You can be drawing a blank.,{You can be drawing a blank due to the complex...
274825,entailment,Houseboats are a beautifully preserved traditi...,The tradition of houseboats originated while t...,{Houseboats were invented by the British in In...


In [16]:
# temp = []
# for index1 in tqdm(range(len(new_dataset_entail))):
#     # print(new_dataset_entail.iloc[index1]['sentence1'])
#     for index2 in range(len(dataset_not_entail)):
#         if new_dataset_entail.iloc[index1]['sentence1'] == dataset_not_entail.iloc[index2]['sentence1']:
#             temp.append(dataset_not_entail.iloc[index2]['sentence2'])
#             break

In [17]:
# print(len(temp))

In [18]:
# new_dataset_entail['negative'] = temp
# new_dataset_entail

In [19]:
new_dataset_entail = new_dataset_entail[new_dataset_entail['negative'].notna()]

In [20]:
new_dataset_entail 

,gold_label,sentence1,sentence2,negative
0,entailment,you know during the season and i guess at at y...,You lose the things to the following level if ...,{They never perform recalls on anything.}
1,entailment,you know during the season and i guess at at y...,You lose the things to the following level if ...,{The recalls are done every Friday.}
2,entailment,One of our number will carry out your instruct...,A member of my team will execute your orders w...,{We have no one free at the moment so you have...
3,entailment,One of our number will carry out your instruct...,A member of my team will execute your orders w...,{You provided concise instructions with clear ...
4,entailment,How do you know? All this is their information...,This information belongs to them.,{We are not sure if they are correct.}
...,...,...,...,...
274822,entailment,If you would like to scale the highest heights...,Some lessons and advice are a good idea if you...,{Climbing is a good sport for novices. }
274823,entailment,Drawing a blank?,You can be drawing a blank.,{You aren't drawing a blank.}
274824,entailment,Drawing a blank?,You can be drawing a blank.,{You can be drawing a blank due to the complex...
274825,entailment,Houseboats are a beautifully preserved traditi...,The tradition of houseboats originated while t...,{Houseboats were invented by the British in In...


In [21]:
new_dataset = Dataset.from_pandas(new_dataset_entail)
new_dataset = new_dataset.rename_column("sentence1", "anchor")
new_dataset = new_dataset.rename_column("sentence2", "positive")
new_dataset = new_dataset.remove_columns(["__index_level_0__"])
new_dataset

Dataset({
    features: ['gold_label', 'anchor', 'positive', 'negative'],
    num_rows: 274825
})

In [22]:
def convert_dataset(example):
    # print(example['negative'])
    number = len(example['negative'][0])
    # print(gold_label)
    gold_label = example['gold_label'] * number
    anchor = example['anchor'] * number
    positive = example['positive'] * number
    return {'gold_label': gold_label, 'anchor': anchor, 
            'positive': positive, 'negative': example['negative'][0]}

        
new_dataset = new_dataset.map(
    lambda example: convert_dataset(example),                   
    remove_columns=['gold_label', 'anchor', 'positive', 'negative'],
    batched=True,
    batch_size=1
)

Map:   0%|          | 0/274825 [00:00<?, ? examples/s]

KeyboardInterrupt: 

In [ ]:
new_dataset

Dataset({
    features: ['gold_label', 'anchor', 'positive', 'negative'],
    num_rows: 24536
})

In [ ]:
print(['']*5)

['', '', '', '', '']


In [ ]:
def add_columns(examples):
    # print(len(examples))
    length = len(examples['gold_label'])
    examples['negative'] = [''] * length
    return examples

In [ ]:
converted_dataset = converted_dataset.rename_column("sentence1", "anchor")
converted_dataset = converted_dataset.rename_column("sentence2", "positive")

In [ ]:
converted_dataset = converted_dataset.map(add_columns, batched=True)

In [ ]:
all_data = DatasetDict()
all_data['train'] = new_dataset
all_data['validation'] = converted_dataset['dev']
all_data['test'] = converted_dataset['test']

In [ ]:
all_data.push_to_hub(f"{dataset_name}_{label}")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/25 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/606 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/presencesw/vinli_3_label_entailment/commit/c8ca77e7edf8b047fc49b633446c1106d3437d81', commit_message='Upload dataset', commit_description='', oid='c8ca77e7edf8b047fc49b633446c1106d3437d81', pr_url=None, pr_revision=None, pr_num=None)